# Creating a conda package from a Modelica Model
This example illustrates how to develop, run, test(, repeat) a Modelica model in a (local) pandas/pyspark environment that can be deployed in a cluster as a conda package for large scale analytics.
- Build and compile the model once
- Make it a conda package
- Call the compiled model with different input data
- Use it on scale. Change parameters only during runtime.

See https://openmodelica.org/doc/OpenModelicaUsersGuide/v1.11.0/ompython.html

In [ ]:
import pandas as  pd
import os
import sys
import tempfile
import DyMat

In [ ]:
# Finds the package in the repo instead of the installed one
sys.path.insert(0, '../../src')

## Parametric simulation without OMC

In [ ]:
from ModelicaModels import BouncingBall

In [ ]:
fig, ax = plt.subplots(figsize=(5,3.5))
ts_df  = dymat2pandas(dm, 2, ['h'])
ax.plot(ts_df['time'], ts_df['h'])
plt.tight_layout()

In [ ]:
parameters_var_df = pd.DataFrame(columns=['run_key', 'modifiers'], data=[
    ['r1', ['e=0.7']],
    ['r2', ['e=0.5']],
    ['r3', ['e=0.9']],
])
parameters_var_df

### Sequential execution

In [ ]:
def dymat2pandas(dm, block, names) -> pd.DataFrame:
    ts_df = pd.DataFrame(dm.getVarArray(names)).T
    #ts_df['time'] = dm.abscissa(2)
    ts_df.columns=['time'] + names
    return ts_df


def run_sim(parameters, res_vars=None) -> pd.DataFrame:
    """Simulation of a single run. The unique run identifier is in the column run_key."""
    mod = BouncingBall.instantiatemodel()
    log_str = ''
    print(parameters)
    temp_dir = tempfile.gettempdir()
    grp = parameters['run_key'].iloc[0]
    resfilename = grp + '.mat'
    resfilepathname = os.path.join(temp_dir, resfilename)
    mod.setParameters(parameters['modifiers'])
    log_str += str(mod.simulate(resultfile=resfilepathname, simflags=None))
    # Collect results
    if isinstance(res_vars, tuple):
        res_vars = list(res_vars)
    try:
        dm = DyMat.DyMatFile(resfilepathname)
        ts_df = dymat2pandas(dm, 2, res_vars)
        os.remove(resfilepathname)
    except Exception as e:
        ts_df = pd.DataFrame(columns=['time'] + res_vars, data=[[-1.0 ,0.0, 0.0]])
    ts_df.columns = ['time'] + res_vars
    ts_df['run_key'] = grp
    return ts_df


def get_sim_dist_func(res_vars=None):
    """Return the pandas (udf) function to simulate a set of runs."""
    def run_sim_dist(parameters) -> pd.DataFrame():
        return run_sim(parameters, res_vars=res_vars)
    return run_sim_dist


In [ ]:
ts_all_df = parameters_var_df.groupby(['run_key']).apply(
        get_sim_dist_func(res_vars=['h', 'v'])
    )

In [ ]:
ts_all_df

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
p_df = ts_all_df[['run_key', 'h']].reset_index(drop=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
for label, df in p_df.groupby('run_key'):
    df['h'].plot(ax=ax, label=label)
plt.legend()

### Parallel execution with Spark

In [ ]:
parameters_var_sdf = spark.createDataFrame(parameters_var_df)
parameters_var_sdf.show()

### Parallel execution

In [ ]:
from TestBouncingBall import run_bouncingball_spark

In [ ]:
ts_sim_sdf = run_bouncingball_spark(parameters_var_sdf)

In [ ]:
# ModuleNotFoundError: No module named 'OMPython'
# Reason: The import is done from the worker, which is probably the desired behavior
ts_sim_sdf.show()

In [ ]:
T.StructType

In [ ]:
from pyspark.sql import functions as F, types as T

In [ ]:
res_schema = T.StructType([ \
    T.StructField("time", T.DoubleType(),True), \
    T.StructField("h", T.DoubleType(),True), \
    T.StructField("v", T.DoubleType(),True), \
    T.StructField("run_key", T.StringType(), True), \
  ])

In [ ]:
mod = ModelicaSystem(
        fileName=instantiatemodel.fn, modelName="BouncingBall",
        xmlFileName=instantiatemodel.xmlfn
    )

ts_all_sdf = parameters_var_sdf.groupby('run_key').applyInPandas(
    get_sim_dist_func(mod, res_vars=('h', 'v')), schema=res_schema
).cache()

In [ ]:
ts_all_sdf.count()

In [ ]:
ts_all_sdf.show()

In [ ]:
mod.getContinuous()

# Old interface
This may allow to build a model and run later...

In [ ]:
from OMPython import OMCSessionZMQ
omc=OMCSessionZMQ()

In [ ]:
omc.sendExpression("getVersion()")

In [ ]:
#omc.sendExpression("loadFile(getInstallationDirectoryPath() + \"/share/doc/omc/testmodels/BouncingBall.mo\")")
omc.sendExpression("loadFile(\"BouncingBall.mo\")")

In [ ]:
omc.sendExpression("instantiateModel(BouncingBall)")

In [ ]:
omc.sendExpression("getClassNames()")

In [ ]:
omc.sendExpression("checkModel(BouncingBall)")

In [ ]:
omc.sendExpression("checkSettings()")

In [ ]:
%%time
omc.sendExpression("buildModel(BouncingBall)")

In [ ]:
#%%time
#omc.sendExpression("simulate(BouncingBall, stopTime=3.0)")

Kann man ein binary bauen und dann mit Optionen und Input-Daten aufrufen?

In [ ]:
os.system('./BouncingBall')

In [ ]:
#omc.sendExpression('buildModel(BouncingBall.mo,BouncingBall)')

In [ ]:
#omc.sendExpression('simulate(my_library.my_model, simflags="-overrideFile=parameter_sweep1.txt", stopTime=86400)')
omc.sendExpression('simulate(BouncingBall.mo, stopTime=2, resultFile=d.mat)')

In [ ]:
vars = omc.sendExpression("readSimulationResultVars(\"BouncingBall_res.mat\",true)")

In [ ]:
vars_str = ','.join(vars)
vars_str

In [ ]:
#data = omc.sendExpression("readSimulationResult(\"BouncingBall_res.mat\",{test.co_sim.A.h})")
#data = omc.sendExpression("readSimulationResult(\"BouncingBall_res.mat\",{time, h})")
data = omc.sendExpression("readSimulationResult(\"BouncingBall_res.mat\",{" + vars_str + "})")

In [ ]:
ts_df = pd.DataFrame(data).T

ts_df.columns = vars
ts_df